In [1]:
'''
目前有如下几份20220831的数据：
1. 原始特征数据
2. label数据
3. 复推标签数据
4. order time数据

分别读取这几份数据，各自去重，然后合并
'''
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA


In [38]:
73456 + 6108

79564

In [39]:
79564 + 2769

82333

In [3]:
# 原始特征数据
raw_feature_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# label数据
label_data = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# 复推标签数据
rerec_data = pd.read_csv('../data_20220831/sample_resume_type_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# order time数据
order_time_data = pd.read_pickle('../generalization_data/cv_jd_time_20220831_82333.pkl')


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/1961729654.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_feature_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/1961729654.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  label_data = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)


In [5]:
print('{}的shape: {}'.format('raw_feature_data', raw_feature_data.shape))
print('{}的shape: {}'.format('label_data', label_data.shape))
print('{}的shape: {}'.format('rerec_data', rerec_data.shape))
print('{}的shape: {}'.format('order_time_data', order_time_data.shape))

raw_feature_data的shape: (82333, 86)
label_data的shape: (82333, 128)
rerec_data的shape: (82333, 3)
order_time_data的shape: (82333, 5)


In [16]:
label_data.columns

Index(['cv_id', 'jd_id', 'cv_code', 'jd_code', 'last_opt', 'da_label',
       'rank_label', 'title', 'deadline', 'category_id',
       ...
       'industry_cut', 'industry_jd2vec', 'industry_vec2', 'majorName_cut',
       'majorName_jd2vec', 'majorName_vec2', 'skills_cut', 'skills_jd2vec',
       'skills_vec2', 'ds'],
      dtype='object', length=128)

In [13]:
rerec_data.columns

Index(['cv_id', 'jd_id', 'resume_type'], dtype='object')

In [14]:
order_time_data.columns

Index(['cv_id', 'jd_id', 'pair_created_time', 'day', 'month'], dtype='object')

In [17]:
# 合并上述表的关键信息，raw feature暂时不用考虑
cv_jd_raw_label_rank_label = label_data[['cv_id', 'jd_id', 'da_label', 'rank_label']]
cv_jd_resume_type = rerec_data[['cv_id', 'jd_id', 'resume_type']]
cv_jd_pair_time = order_time_data[['cv_id', 'jd_id', 'month']].rename(columns={'month': 'order_month'})

In [21]:
res = cv_jd_raw_label_rank_label.\
            join(cv_jd_resume_type.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
            join(cv_jd_pair_time.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
res

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,1,1,NaN,2021-09
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,-1,1,4.0,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,1,1,4.0,2022-03
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,2,2,NaN,2021-10
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,3,3,NaN,2021-10
...,...,...,...,...,...,...
82328,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,-1,1,5.0,2022-08
82329,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,-1,2,4.0,2022-07
82330,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,3,3,4.0,2022-08
82331,8a69d7c282199453018247bb00cb68a3,8a69f6f8823a93b9018247c30aca4e3d,-1,2,4.0,2022-07


In [32]:
# 取出复推集
rerec_cvjd = res[(res['resume_type']==6)]
rerec_cvjd.to_pickle('../generalization_data/cvjd_rerec_2769.pkl')
rerec_cvjd

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
225,2c920715802399a801802c857e7c161f,2c920715800e886301804fe3adcf13aa,0,0,6.0,2022-05
295,2c9207158059e82b018072402f4a6328,2c920715800e886301805fd9d4016644,0,0,6.0,2022-05
296,2c9207158059e82b018072402f4a6328,2c9207157f347800017f87cacec42e72,0,0,6.0,2022-05
301,2c9207158059e82b0180970f82eb1251,2c920715800e886301805aff59901824,0,0,6.0,2022-05
303,2c9207158059e82b0180970f82eb1251,2c920715800e886301805aef83e96eb7,0,0,6.0,2022-05
...,...,...,...,...,...,...
82239,2c920715811ee1b90181215203ad644c,2c92071580f6d9350180fa6dd85d779a,0,0,6.0,2022-06
82275,2c92071581ba3b190181c85bb79c31ff,2c9207158114fb2901815c2baf0d30ed,0,0,6.0,2022-07
82286,2c92071581d380c20181dfe95dac720d,2c92071581cdd33b0181d30032077a68,1,1,6.0,2022-07
82291,2c92071581d380c20181fbbb8c003c9f,2c92071581afaf5b0181b410e2390f40,0,0,6.0,2022-07


In [33]:
# 去除复推集后，取出2022-08的数据
cvjd_202208 = res[~(res['resume_type']==6)][res['order_month']=='2022-08']
cvjd_202208.to_pickle('../generalization_data/cvjd_202208_6108.pkl')
cvjd_202208

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/3225369999.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cvjd_202208 = res[~(res['resume_type']==6)][res['order_month']=='2022-08']


,cv_id,jd_id,da_label,rank_label,resume_type,order_month
470,8a69c41582926ae60182a07b24c1048c,8a69f62f82926a8601829f1379a07648,2,2,4.0,2022-08
471,8a69c41582926ae60182a0e09bf27664,8a69d77e82926b410182a083fbf602c0,3,3,4.0,2022-08
472,8a69c41582926ae60182a119de713808,8a69f68782590ecb01826326b5e94919,-1,2,4.0,2022-08
473,8a69c41582a3cc240182b0354c0e7bad,2c92071581ebf3390181ec9e500755e4,1,1,4.0,2022-08
474,8a69c4a3827cc886018284b9612d0f03,8a69f601823a92b0018243dcb9c45f7a,-1,1,4.0,2022-08
...,...,...,...,...,...,...
82316,8a69d71b82add4e10182aebff7d016d2,8a69c468826ee9b4018290a64a0c01bb,2,2,4.0,2022-08
82325,8a69d7b4826d7dfb01826d7efad200f1,2c920715818a462a01818e3f9c0a2337,3,3,4.0,2022-08
82328,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,-1,1,5.0,2022-08
82330,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,3,3,4.0,2022-08


In [37]:
# 去除复推集，去除202208的数据，剩下的数据就是train和test的数据了
cvjd_traintest = res[~(res['resume_type']==6)][~(res['order_month']=='2022-08')]

cvjd_traintest.to_pickle('../generalization_data/cvjd_traintest_73456.pkl')
cvjd_traintest

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/338758848.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cvjd_traintest = res[~(res['resume_type']==6)][~(res['order_month']=='2022-08')]


,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,1,1,NaN,2021-09
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,-1,1,4.0,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,1,1,4.0,2022-03
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,2,2,NaN,2021-10
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,3,3,NaN,2021-10
...,...,...,...,...,...,...
82324,8a69d7a68219933d01823d93f8022296,2c92071581ebf3390182009a771b5b07,1,1,4.0,2022-07
82326,8a69d7c2821994530182334ac6bb128e,8a69c4a8820f0ee00182143af21d26f8,-1,2,4.0,2022-07
82327,8a69d7c2821994530182386fdad44a80,8a69f67f82198d89018223bca0291137,3,3,4.0,2022-07
82329,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,-1,2,4.0,2022-07


In [35]:
73456+6108

79564

In [58]:
# 训练集和测试集需要做一次过滤
cvjd_traintest_filter = cvjd_traintest[~((cvjd_traintest['da_label']==-1)&(cvjd_traintest['rank_label']==1))]
cvjd_traintest_filter.to_pickle('../generalization_data/cvjd_traintest_filter_67925.pkl')

cvjd_traintest_filter

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,1,1,NaN,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,1,1,4.0,2022-03
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,2,2,NaN,2021-10
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,3,3,NaN,2021-10
5,2c9207157c9f1919017cc065bd545ef8,2c9207157ccbcd86017d08622b3e37f7,2,2,4.0,2021-11
...,...,...,...,...,...,...
82324,8a69d7a68219933d01823d93f8022296,2c92071581ebf3390182009a771b5b07,1,1,4.0,2022-07
82326,8a69d7c2821994530182334ac6bb128e,8a69c4a8820f0ee00182143af21d26f8,-1,2,4.0,2022-07
82327,8a69d7c2821994530182386fdad44a80,8a69f67f82198d89018223bca0291137,3,3,4.0,2022-07
82329,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,-1,2,4.0,2022-07


In [42]:
cvjd_traintest_filter['rank_label'].value_counts()

1    23704
2    19888
0    16230
3     8103
Name: rank_label, dtype: int64

# 训练集和测试集切分

In [45]:
train_test_data_filter_0 = cvjd_traintest_filter[cvjd_traintest_filter['rank_label']==0]
train_test_data_filter_1 = cvjd_traintest_filter[cvjd_traintest_filter['rank_label']==1]
train_test_data_filter_2 = cvjd_traintest_filter[cvjd_traintest_filter['rank_label']==2]
train_test_data_filter_3 = cvjd_traintest_filter[cvjd_traintest_filter['rank_label']==3]

In [50]:
from sklearn.model_selection import train_test_split
'''
train_test_data_filter_0 rank_label=0 数量：16230
train_test_data_filter_1 rank_label=1 数量：23704
train_test_data_filter_2 rank_label=2 数量：19888
train_test_data_filter_3 rank_label=3 数量：8103
'''
loop_list = [train_test_data_filter_0, train_test_data_filter_1, train_test_data_filter_2, train_test_data_filter_3]

sample_train = None
sample_test = None
for data in loop_list:
    train, test = train_test_split(data, test_size=0.2, random_state=10)
    sample_train = pd.concat([sample_train, train])
    sample_test = pd.concat([sample_test, test])

In [60]:
sample_train.to_pickle('../generalization_data/cvjd_train_filter_54339.pkl')

sample_train['rank_label'].value_counts()
sample_train.shape

(54339, 6)

In [61]:
sample_test.to_pickle('../generalization_data/cvjd_test_filter_13586.pkl')

sample_test['rank_label'].value_counts()
sample_test.shape

(13586, 6)